# Link to tutorial: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [1]:
%matplotlib inline
import argparse
import os
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vision_utils

from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Make Notebook reproducable

In [2]:
random.seed(1337)
torch.manual_seed(1337)
torch.use_deterministic_algorithms(True)

# Define Training Hyperparameters

In [10]:
data_root: str = "data/celeba"
num_workers: int = 2
batch_size: int = 128
image_size: int = 64
num_channels: int = 3
len_z_vector: int = 100
depth_feature_map_discriminator: int = 64
depth_feature_map_generator: int = 64
num_epochs: int = 5
learning_rate: float = 0.0002
beta_1: float = 0.5
num_gpus: int = 1

# Data Loading

In [11]:
dataset: datasets.ImageFolder = datasets.ImageFolder(
    root=data_root,
    transform=transforms.Compose(
        [
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]
    )
)

dataloader: DataLoader = DataLoader(dataset=dataset, 
                                    batch_size=batch_size,
                                    shuffle=True,
                                    num_workers=num_workers)

device = torch.device("cuda" if (torch.cuda.is_available() and num_gpus > 0) else "cpu")

real_batch_images = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vision_utils.make_grid(real_batch_images[0].to(device)[:64], padding=2, normalize=True).cpu(), (1,2,0)))

FileNotFoundError: [Errno 2] No such file or directory: 'data/celeba'

# Init Weights

In [5]:
def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv") != 1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0)

# Build Generator

In [14]:
class Generator(nn.Module):
    def __init__(self, num_gpus):
        super(Generator, self).__init__()
        self.num_gpus = num_gpus
        self.model = nn.Sequential(
            # Input Vector Z to Conv
            nn.ConvTranspose2d(len_z_vector, depth_feature_map_generator * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(depth_feature_map_generator * 8),
            nn.ReLU(True),
            # Upsample
            nn.ConvTranspose2d(depth_feature_map_generator * 8, depth_feature_map_generator * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(depth_feature_map_generator * 4),
            nn.ReLU(True),
            # Upsample
            nn.ConvTranspose2d(depth_feature_map_generator * 4, depth_feature_map_generator * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(depth_feature_map_generator * 2),
            nn.ReLU(True),
            # Upsample
            nn.ConvTranspose2d(depth_feature_map_generator * 2, depth_feature_map_generator, 4, 2, 1, bias=False),
            nn.BatchNorm2d(depth_feature_map_generator),
            nn.ReLU(True),
            # Final Upsample
            nn.ConvTranspose2d(depth_feature_map_generator, num_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.model(x)
        

In [15]:
generator: Generator = Generator(num_gpus).to(device)

if device.type == "cuda" and num_gpus > 1:
    generator: Generator = nn.DataParallel(generator, list(range(num_gpus)))

generator.apply(init_weights)

print(generator)

NameError: name 'device' is not defined